In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
a = .1
sigma = .01
r_0 = .025
dt = 1/252
T_max = 5.0
time_grid = np.arange(0, T_max, dt)
n_paths = 10000

In [ ]:
from dataclasses import dataclass

@dataclass
class HullWhiteParams:
  a: float
  sigma: float
  r_0: float
  dt: float
  T: int
  n_paths: int
  time_grid: np.ndarray


  def zero_rate(self, T):
    return 0.02 + 0.005 * (1-np.exp(-0.3*T))
  
  def P0T(self, T):
    return np.exp(-self.zero_rate(T))

  def f0T(self, T=None):
    eps = np.finfo(float).eps
    if not T:
      T = self.T
    return -(np.log(self.P0T(T + eps)) 
                - np.log(self.P0T(T - eps))) / (2 * eps)

  def df0T_dt(self, t = None):
    eps = np.finfo(float).eps
    if not t:
      t = self.T
    return (self.f0T(t + eps) - self.f0T(t - eps)) / (2 * eps)

  def theta(self, t):
    return (
              self.df0T_dt(t) + self.a*self.f0T(t) + 
              self.sigma**2 / 2*a**2*(1-np.exp(-2*a*t))
           )
     


In [ ]:
class HullWhiteSimulation(HullWhiteParams):
  def __init_(self):
    self.r_i = None

  def trapezoidal_approx_phi(self, t):
    phi = self.dt * (self.theta(t) * np.exp(-a*self.dt)+ self.theta(t+self.dt))
    return phi/2

  def exact_short_rate(self):
    size = (self.n_paths, self.time_grid.shape[0])
    r_i = np.zeros(size)
    r_i[:, 0] = self.r_0
    Z = np.random.standard_normal(size=size)
    for i in range(1, self.time_grid.shape[0]):
      r_i[:, i] = r_i[:, i-1] + self.trapezoidal_approx_phi(self.time_grid[i-1])
      r_i[:, i] += np.sqrt(self.sigma**2/(2*self.a)*(1-np.exp(-2*self.a*self.dt))) * Z[:, i]
    self.r_i = r_i
    return r_i

  def plot_simulation(self):
    assert self.r_i is not None
    plt.plot(self.r_i.T)
    plt.show()

  


In [ ]:
hullwhite = HullWhiteSimulation(a, sigma, r_0, dt, T_max, n_paths, time_grid)

In [ ]:
hullwhite.exact_short_rate()
hullwhite.plot_simulation()